<div style="margin: 0px; padding: 10px; background-color: #43a047;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px;">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white; text-align:center">🔥⚡ Blazing Fast EDA - Coleridge + Baseline Model ⚡🔥</h1>
      
<!--       <div style="color:white">I am a square root.</div> -->
<!--       <h1>$\sqrt{4}$</h1> -->
  </div>
</div>

<div style="margin: 0px; padding: 10px; background-color: #00897b;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px;">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white; text-align:center">Table of Contents</h1>
  </div>
  
  <div style="margin: 0px; padding: 10px; background-color: #ffffff;
        box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
        border-radius:2px; text-align:left">
    <h5><a href="#0">0. Reference Notebooks</a></h5>
    <h5><a href="#1">1. Importing the required libraries</a></h5>
    <h5><a href="#2">2. Modin makes all the difference</a></h5>
    <h5><a href="#3">3. Data Exploration and Acquisition</a></h5>
    <h5><a href="#4">4. Data Preprocessing and Cleaning</a></h5>
    <h5><a href="#5">5. Data Visualization</a></h5>
    <h5><a href="#6">6. Baseline Model</a></h5>
    <h5><a href="#7">7. Submission</a></h5>
    <h5><a href="#8">8. Conclusion</a></h5>
    
  </div>
</div>

<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px;">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white; text-align:center">0. Reference Notebooks 📓</h1>
  </div>
  
  <div style="margin: 0px; padding: 10px; background-color: #ffffff;
        box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
        border-radius:2px; text-align:left">
    <h5><a href="https://www.kaggle.com/prashansdixit/coleridge-initiative-eda-baseline-model">1. 📝Coleridge Initiative-EDA📚 & Baseline Model🎯</a></h5>
  </div>
</div>

<a id='1'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px;
            text-align:center">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white">1. Importing the required libraries 📗</h1>
  </div>
</div>

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

import os
import re
import json
import glob
import subprocess
import sys
from collections import defaultdict
from textblob import TextBlob
from functools import partial

import numpy as np # linear algebra

# visualization libraries
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

# NLP libraries
import nltk
import spacy
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])
nlp.max_length = 4000000
from nltk.probability import FreqDist
from wordcloud import WordCloud, STOPWORDS
import string

from tqdm.autonotebook import tqdm

<a id='2'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white;text-align:center">2. <b>Modin</b> makes all the difference</h1>
  </div>
  <br>
  <div style="color:white"><b>Modin</b> is a data processing library similar to <b>Pandas</b>. However it runs on all the cores available on the machine, unlike Pandas which runs only on a single core. <b>Modin</b> uses <b>Ray</b> or <b>Dask</b> behind the scenes for parallel computation. Using <b>Modin</b> significantly improves the code execution time as compared to the reference notebook.</div>
</div>

In [ ]:
# Uncomment and install in your Kaggle working directory.
!pip install ../input/pythonmodin/modin-0.9.1-py3-none-manylinux1_x86_64.whl

In [ ]:
import modin.pandas as pd

In [ ]:
base_path = '/kaggle/input/coleridgeinitiative-show-us-the-data/'

# reading csv files and train & test file paths
train_df = pd.read_csv(base_path + 'train.csv')
sample_sub_df = pd.read_csv(base_path + 'sample_submission.csv')
train_files_path = base_path + 'train/'
test_files_path = base_path + 'test/'

<a id='3'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px;
            text-align:center">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white">3. Data Exploration and Acquisition 🔍</h1>
  </div>
</div>

In [ ]:
train_df.head()

In [ ]:
train_df.info()

<div style="margin: 0px; padding: 10px; background-color: #ef6c00;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px;
            text-align:center">
  <div style="margin: 0; padding: 0; width: 100%">
      <div style="color:white">The dataset has no null values</div>
  </div>
</div>

In [ ]:
#finding unique values in each column
[(f"{col}:{len(train_df[col].unique())}") for col in train_df.columns]

In [ ]:
def get_text_from_json_file(filename, folder_path=train_files_path, select_only='all'):
    '''
    Function to get the text from the json file and return it. This will be used to append the returned text 
    to the dataframe
    '''
    json_file_path = os.path.join(folder_path, filename+'.json')
    headings_list = []
    body_list = []
    combined_list = []
    
    with open(json_file_path, 'r') as fhandle:
        json_str = json.load(fhandle)
        
        for item in json_str:
            headings_list.append(item.get('section_title'))
            body_list.append(item.get('text'))
            combined_list.append(item.get('section_title'))
            combined_list.append(item.get('text'))
    
    all_headings = ' '.join(headings_list)
    all_body = ' '.join(body_list)
    all_combined = ' '.join(combined_list)
    
    if select_only == 'all':
        return all_combined
    elif select_only == 'heading':
        return all_headings
    elif select_only == 'body':
        return all_body

In [ ]:
%%time
train_df['text'] = train_df['Id'].apply(get_text_from_json_file)

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
%%time
sample_sub_df['text'] = sample_sub_df['Id'].apply(partial(get_text_from_json_file, folder_path=test_files_path))

In [ ]:
sample_sub_df

In [ ]:
def preprocess_text(text):
    
    '''
    This function converts the input text to lower case, then removes punctuation, emojis and 
    multiple spaces and finally performs lemmatization
    
    text - Sentence that needs to be cleaned
    '''
    
    text = ''.join([k for k in text if k not in string.punctuation])
    text = re.sub('r[^\w\s]', ' ', str(text).lower()).strip()
    lem = nltk.stem.wordnet.WordNetLemmatizer()
    text = lem.lemmatize(text)
    
    return text

In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower().strip())

<a id='4'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px;
            text-align:center">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white">4. Data Preprocessing and Cleaning 🧹</h1>
  </div>
</div>

In [ ]:
%%time
train_df['text'] = train_df['text'].apply(preprocess_text)

In [ ]:
%%time 
sample_sub_df['text'] = sample_sub_df['text'].apply(preprocess_text)

<a id='5'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px;
            text-align:center">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white">5. Data Visualization 📊</h1>
  </div>
</div>

In [ ]:
words = list(train_df['cleaned_label'].values)
stop_words = ['ourselves', 'hers','the','of','and','in', 'between', 'yourself', 'but', 'again','of', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']
split_words = []
for word in words:
    lo_w = []
    list_of_words = str(word).split()
    for w in list_of_words:
        if w not in stop_words:
            lo_w.append(w)
    split_words.append(lo_w)
all_words = []
for word_list in split_words:
    all_words += word_list

In [ ]:
mostcommon_small = FreqDist(all_words).most_common(30)
x, y = zip(*mostcommon_small)
plt.figure(figsize=(50,30))
plt.margins(0.02)
plt.bar(x, y)
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.tight_layout(pad=0)
plt.title('Freq of 30 Most Common Words in cleaned_label', fontsize=60)
plt.show()

most_common_words = FreqDist(all_words).most_common(100)
word_cloud = WordCloud(width=1600, height=800, background_color='white', stopwords=STOPWORDS).generate(str(most_common_words))

fig = plt.figure(figsize=(30,10), facecolor='white')
plt.imshow(word_cloud, interpolation="bilinear")
plt.axis('off')
plt.title('Top 100 Most Common Words in cleaned_label', fontsize=50)
plt.tight_layout(pad=0)
plt.show()

<a id='6'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px;
            text-align:center">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white">6. Baseline Model 📈</h1>
  </div>
</div>

In [ ]:
temp_1 = [x.lower() for x in train_df['dataset_label'].unique()]
temp_2 = [x.lower() for x in train_df['dataset_title'].unique()]
temp_3 = [x.lower() for x in train_df['cleaned_label'].unique()]

existing_labels = set(temp_1 + temp_2 + temp_3)

In [ ]:
id_list = []
predictions_list = []

for index, row in tqdm(sample_sub_df.iterrows()):
#     print('\n\n\n')
#     print(index, row, '-'*50)
    sample_text = row['text']
    
    row_id = row['Id']
    temp_df = train_df[train_df['Id'] == row['Id']]
#     print('PRINTING TEMP_DF')
#     print(temp_df.head())
    
#     print('SAMPLE TEXT')
#     print(sample_text)
#     print('TEMP_DF TEXT')
#     print(temp_df['text'][0])
    
    cleaned_labels = temp_df['cleaned_label'].to_list()
    for known_label in existing_labels:
        if known_label in sample_text.lower():
            cleaned_labels.append(clean_text(known_label))
    
    cleaned_labels = [clean_text(x) for x in cleaned_labels]
    cleaned_labels = set(cleaned_labels)
    predictions_list.append('|'.join(cleaned_labels))
    id_list.append(row_id)

<a id='7'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px;
            text-align:center">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white">7. Submission 📜</h1>
  </div>
</div>

In [ ]:
submission = pd.DataFrame()
submission['Id'] = id_list
submission['PredictionString'] = predictions_list
submission.head()

In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
!cd /kaggle/working/ && ls

<a id='8'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
  <div style="margin: 0; padding: 0; width: 100%; text-align:center">
      <h1 style="color:white">8. Conclusion 🎁</h1>
  </div>
  <br>
  <div style="color:white">This notebook uses basic string matching to match the names of the datasets mentioned in the research papers. Further improvements can be made where the model generalizes and identifies the datasets not present in the given list. <br> And that is a work in progress 😃 </div>
</div>

<div style="margin: 0px; padding: 10px; background-color: #43a047;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px;">
  <div style="margin: 0; padding: 0; width: 100%">
      <h2 style="text-align:center; color:white">
          Voila!!! You have reached the end of this kernel. Feel free to fork this kernel. If you find this notebook useful, then please upvote and leave your valuable comments. Thank you. 🙏
      </h2>
      <br>
      <h4 style="color:white; text-align:center">Created by Kartik B Bhargav</h4>

  </div>
</div>